In [0]:
import os
import cv2
import matplotlib.pyplot as plt

# New Section

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
datadir = "/content/drive/My Drive/Flowers"
valdir = "/content/drive/My Drive/Flowers_valid"
categories = ["Angels Trumpet","Beach Moonflower","Bougainvillea","Cyclamen","Daisy","Dandelion","Foxgloves","Frangipani","Gladiolus","Hawaiian Hibiscus","Nasturtium","Purple Passionflower","Rose","Sacred Lotus","Sunflower","Tulip","Wallflowers"]
img_size = 200

In [0]:

from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f010c41a3c8> False
<keras.layers.convolutional.Conv2D object at 0x7f00bf2be390> False
<keras.layers.convolutional.Conv2D object at 0x7f00bf2be240> False
<keras.layers.pooling.MaxPooling2D object at 0x7f00bf269b00> False
<keras.layers.convolutional.Conv2D object at 0x7f00bf269438> False
<keras.layers.convolutional.Conv2D object at 0x7f00bea196a0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f00bea32f60> False
<keras.layers.convolutional.Conv2D object at 0x7f00bea32c18> False
<keras.layers.convolutional.Conv2D object at 0x7f00be9dca90> False
<keras.layers.convolutional.Conv2D object at 0x7f00be982240> False
<keras.layers.pooling.MaxPooling2D object at 0x7f00be9a0ac8> False
<keras.layers.convolutional.Conv2D object at 0x7f00be9a0630> False
<keras.layers.convolutional.Conv2D object at 0x7f00be94b2e8> False
<keras.layers.convolutional.Conv2D object at 0x7f00be961d30> False
<keras.layers.pooling.MaxPooling2D object at 0x7f00be90c5c0>

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 5125      
Total params: 33,595,205
Trainable params: 25,959,941
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 


In [0]:
train_data_dir = 'content/drive/My Drive/Flowers'
validation_data_dir = '/content/drive/My Drive/Flowers_valid'

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2, 
      horizontal_flip=True,
      fill_mode='nearest')

In [0]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_batchsize = 100
val_batchsize = 10

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/30
35/34 [==============================] - 1009s 29s/step - loss: 1.2847 - acc: 0.4993 - val_loss: 0.6040 - val_acc: 0.7764
Epoch 2/30
35/34 [==============================] - 49s 1s/step - loss: 0.6836 - acc: 0.7392 - val_loss: 0.9874 - val_acc: 0.7011
Epoch 3/30
35/34 [==============================] - 57s 2s/step - loss: 0.5907 - acc: 0.7862 - val_loss: 0.4350 - val_acc: 0.8516
Epoch 4/30
35/34 [==============================] - 56s 2s/step - loss: 0.4354 - acc: 0.8378 - val_loss: 0.4042 - val_acc: 0.8589
Epoch 5/30
35/34 [==============================] - 56s 2s/step - loss: 0.3936 - acc: 0.8573 - val_loss: 0.3652 - val_acc: 0.8767
Epoch 6/30
35/34 [==============================] - 55s 2s/step - loss: 0.3586 - acc: 0.8675 - val_loss: 0.3582 - val_acc: 0.8903
Epoch 7/30
35/34 [==============================] - 53s 2s/step - loss: 0.3187 - acc: 0.8839 - val_loss: 0.3290 - val_acc: 0.8777
Epoch 8/30
35/34 [==============================] - 53s 2s/step - loss: 0.2715 - acc: 0

In [0]:
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'model.pbs' ) # Your model's name
converter.post_training_quantize=True
model = converter.convert()
file = open( 'q_model.tflite' , 'wb' ) 
file.write( model )

In [0]:
def keras_to_tensorflow(keras_model, output_dir, model_name,out_prefix="output_", log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], out_prefix + str(i + 1))

    sess = K.get_session()

    from tensorflow.python.framework import graph_util, graph_io

    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, name=model_name, as_text=False)

    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard

        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)


In [0]:
keras_to_tensorflow(model,output_dir=output_dir,model_name="saved_model.pb")